# 0. 준비

In [4]:
# 라이브러리를 불러옵니다.
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 1. 불러오기

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains
import time
import json 


def click_button(driver, wait, xpath):
    """주어진 XPath의 버튼을 클릭."""
    button = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
    ActionChains(driver).click(button).perform()


def scrape_content(wait, common_path, relative_loc):
    """모달에서 콘텐츠를 추출."""
    content_dict = {}
    for key, value in relative_loc.items():
        try:
            if key == 'img':
                content_dict[key] = wait.until(EC.presence_of_element_located((By.XPATH, common_path + value))).get_attribute('src')
            else:
                content_dict[key] = wait.until(EC.presence_of_element_located((By.XPATH, common_path + value))).text
        except Exception as e:
            print(f"Error fetching {key}: {e}")
    return content_dict

def extract_elements(driver, content, common_path, elements_path):
    """
    모델에서 여러값을 가진 콘텐츠를 리스트로 추출
    :param content: dict, 콘텐츠가 담김
    :param elements_path: dict, genre와 stars의 여러 요소가 포함된 XPATH가 담겨있음
    """
    
    for key, xpath_template in elements_path.items():
        elements = []
        i = 1
        while True:
            try:
                # 요소를 찾기 위한 전체 XPATH
                xpath = f"{common_path}{xpath_template.format(i)}"
                element = driver.find_element(By.XPATH, xpath)
                elements.append(element.text)
                i += 1
            except Exception:  # 더이상 요소가 발견되지 않으면 멈춤
                break
        content[key] = elements
    
    return content

def main(country_code='KR', top_n=10): # top_n <= 50까지만 문제없이 작동함
    """메인 함수."""
    result = []
    url = f'https://www.imdb.com/search/title/?countries={country_code}'
    common_path = '/html/body/div[4]/div[2]/div/div[2]/div/div'
    relative_loc = {
        'img': '/div[1]/div[1]/div/div/img',
        'title': '/div[1]/div[2]/div[1]/a/h3',
        'year': '/div[1]/div[2]/ul[1]/li[1]',
        'score': '/div[1]/div[2]/div[2]/span/span[1]',
        'summary': '/div[2]'
    }
    

    elements_path ={'genre': '/div[1]/div[2]/ul[2]/li[{}]','stars': '/div[3]/div/ul/li[{}]'}
    
    with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
        driver.get(url)
        wait = WebDriverWait(driver, 20)  # WebDriverWait의 타임아웃 시간을 늘림


        for i in range(1, top_n + 1):
            print(f"Processing item {i}...")
            try:
                # 버튼 클릭
                button_xpath = f'//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li[{i}]/div/div/div/div[1]/div[3]/button'
                click_button(driver, wait, button_xpath)
                time.sleep(0.5)

                # 콘텐츠 크롤링
                content = scrape_content(wait, common_path, relative_loc)
                
                time.sleep(2)

                # content_dict에 국가코드 추가
                content['country'] = country_code

                # content_dict에 장르, 배우 등 list요소 추가 
                content = extract_elements(driver, content, common_path, elements_path)

                # content_dict에 순위 추가
                content['rank'] = i

                result.append(content)

                # 모달 닫기
                close_button_xpath = '/html/body/div[4]/div[2]/div/div[1]/button'
                click_button(driver, wait, close_button_xpath)
                time.sleep(0.5)

            except Exception as e:
                print(f"Error processing item {i}: {e}")
                
    return result


if __name__ == "__main__":
    result = []
    countries = ["KR","KP"]

    for country in countries:
        result.extend(main(country_code=country, top_n=10))
    
    # JSON 파일로 저장
    
    with open('movies_data.json', 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)
    
    print("JSON 파일이 성공적으로 저장되었습니다!")
    


Processing item 1...
Processing item 2...
Processing item 3...
Processing item 4...
Processing item 5...
Processing item 6...
Processing item 7...
Processing item 8...
Processing item 9...
Processing item 10...
Processing item 1...
Processing item 2...
Processing item 3...
Processing item 4...
Processing item 5...
Processing item 6...
Processing item 7...
Processing item 8...
Processing item 9...
Processing item 10...
JSON 파일이 성공적으로 저장되었습니다!


# 더 필요한 크롤링
- 국가별 링크(고유id) 크롤링 -> DB화
- 영화 포스터 링크


## 영화 포스터 링크

# 전처리
- JSON 형식으로
    - 순회할 때마다 저장(리스트에 텍스트로)
    - 텍스트를 개행 단위로 또 리스트에 저장
    - 텍스트 0번째: rank(int), 1번째: title(str), 2번째[:4]: year(list(int)), 3번째: genre(list(str)), 4번째: score(float), 7번쨰: summary(str), 8번째: director(list(str)), 9번째: stars(list(str))
    - 이 규칙이 깨진다면....에러 메시지 출력할 수 있게...
    - genre는 대문자 나오기 직전까지 잘라서 리스트에 저장

- 국가별로 한다면 country(str) 추가.. country에 rank가 종속, title에 나머지가 종속 id를 만들어야 할텐데... id는 summary가 식별가능한 유일한 값인듯..

# 문제
- text를 추출해서 split으로 구분하면 중간에 콘텐츠가 누락되는 경우 오류가 발생하고, 검출하기 어렵다.
# 시도
- 요소별로(제목, 포스터, 연도, 평점, 장르, 줄거리, 배우, 감독) XPATH를 추출해서 text나 src 등 속성을 가져오는 함수를 짠다.
# 해결
- 각각의 요소들의 XPATH를 확인하고, 모달마다 변경되는게 있는지 확인해본 결과 상대 경로는 모두 동일해서 손쉽게 가져올 수 있게 되었다.
# 알게된 점
- 모달은 전체 HTML에서 바로 확인이 불가능하고 javascript 클릭을 통해서만 접근이 가능하다.

In [ ]:
# label과 text를 받아와서 dict로 묶어야 함!!!!!

//*[@id="react-autowhatever-1--item-0"]/div/div/label
//*[@id="react-autowhatever-1--item-1"]/div/div/label

# 테스트코드

In [ ]:
import pytest

In [ ]:
## 각 for문마다 저장